<a href="https://colab.research.google.com/github/lidaprogramer/TransUNet-UNet3D/blob/main/TransUNet_post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify your data and repository paths
import os
print("ImageCAS directory contents:")
print(os.listdir('/content/drive/MyDrive/ImageCAS/'))
print("\nTransUNet repository contents:")
print(os.listdir('/content/drive/MyDrive/TransUNet-UNet3D-main'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ImageCAS directory contents:
['1.label.nii.gz', '2.label.nii.gz', '1.img.nii.gz', '2.img.nii.gz', '4.label.nii.gz', '3.label.nii.gz', '5.label.nii.gz', '3.img.nii.gz', '6.label.nii.gz', '5.img.nii.gz', '7.label.nii.gz', '4.img.nii.gz', '8.label.nii.gz', '6.img.nii.gz', '9.label.nii.gz', '8.img.nii.gz', '7.img.nii.gz', '10.label.nii.gz', '9.img.nii.gz', '11.label.nii.gz', '12.label.nii.gz', '13.label.nii.gz', '10.img.nii.gz', '14.label.nii.gz', '11.img.nii.gz', '14.img.nii.gz', '15.label.nii.gz', '12.img.nii.gz', '13.img.nii.gz', '16.label.nii.gz', '17.label.nii.gz', '15.img.nii.gz', '18.label.nii.gz', '19.label.nii.gz', '16.img.nii.gz', '18.img.nii.gz', '20.label.nii.gz', '17.img.nii.gz', '19.img.nii.gz', '21.label.nii.gz', '22.label.nii.gz', '20.img.nii.gz', '23.label.nii.gz', '24.label.nii.gz', '22.img.nii.gz', '21.img.nii.gz', '25.label.nii.gz', '26.label.

In [3]:
# Change to the repository directory
import os
os.chdir('/content/drive/MyDrive/TransUNet-UNet3D-main')

# Install required packages
!pip install torch torchvision numpy tqdm tensorboard tensorboardX ml-collections medpy SimpleITK scipy h5py opencv-python

# Additional packages that might be needed
!pip install nibabel matplotlib

In [ ]:
# Create necessary directories
import os
from pathlib import Path

# Create model directory for pretrained weights
model_dir = Path('/content/drive/MyDrive/TransUNet-UNet3D-main/model/vit_checkpoint/imagenet21k')
model_dir.mkdir(parents=True, exist_ok=True)

# Create data directory structure
data_dir = Path('/content/drive/MyDrive/TransUNet-UNet3D-main/data/ImageCas')
data_dir.mkdir(parents=True, exist_ok=True)

# Create subdirectories
(data_dir / 'train').mkdir(exist_ok=True)
(data_dir / 'label').mkdir(exist_ok=True)
(data_dir / 'train_processed_224').mkdir(exist_ok=True)
(data_dir / 'val_processed_224').mkdir(exist_ok=True)

# Create lists directory
lists_dir = Path('/content/drive/MyDrive/TransUNet-UNet3D-main/lists/lists_ImageCas')
lists_dir.mkdir(parents=True, exist_ok=True)

print("Directory structure created successfully!")

In [ ]:
import requests
import os

model_path = '/content/drive/MyDrive/TransUNet-UNet3D-main/model/vit_checkpoint/imagenet21k'
model_url = 'https://storage.googleapis.com/vit_models/imagenet21k/R50+ViT-B_16.npz'
model_file = f'{model_path}/R50+ViT-B_16.npz'

if not os.path.exists(model_file):
    print("Downloading pre-trained model...")
    response = requests.get(model_url, stream=True)
    response.raise_for_status()

    with open(model_file, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Model downloaded successfully!")

In [ ]:
drive.mount('/content/drive')


In [7]:
import shutil
import os
from pathlib import Path

# Create local directories
os.makedirs('/content/local_data/train', exist_ok=True)
os.makedirs('/content/local_data/label', exist_ok=True)
os.makedirs('/content/local_data/processed', exist_ok=True)

print("Copying data from Google Drive to local storage...")

# Copy training images
src_train = '/content/drive/MyDrive/TransUNet-UNet3D-main/data/ImageCas/train'
dst_train = '/content/local_data/train'

train_files = [f for f in os.listdir(src_train) if f.endswith('.nii.gz')]
print(f"Found {len(train_files)} training files to copy")

for i, file in enumerate(train_files):
    try:
        shutil.copy2(f'{src_train}/{file}', f'{dst_train}/{file}')
        if (i + 1) % 50 == 0:  # Print progress every 50 files
            print(f"Copied {i + 1}/{len(train_files)} training files")
    except Exception as e:
        print(f"Error copying {file}: {e}")

# Copy label files
src_label = '/content/drive/MyDrive/TransUNet-UNet3D-main/data/ImageCas/label'
dst_label = '/content/local_data/label'

label_files = [f for f in os.listdir(src_label) if f.endswith('.nii.gz')]
print(f"Found {len(label_files)} label files to copy")

for i, file in enumerate(label_files):
    try:
        shutil.copy2(f'{src_label}/{file}', f'{dst_label}/{file}')
        if (i + 1) % 50 == 0:  # Print progress every 50 files
            print(f"Copied {i + 1}/{len(label_files)} label files")
    except Exception as e:
        print(f"Error copying {file}: {e}")

print("Data copying completed!")

Copying data from Google Drive to local storage...
Found 400 training files to copy
Copied 50/400 training files
Copied 100/400 training files
Copied 150/400 training files
Copied 200/400 training files
Copied 250/400 training files
Copied 300/400 training files
Copied 350/400 training files
Copied 400/400 training files
Found 400 label files to copy
Copied 50/400 label files
Copied 100/400 label files
Copied 150/400 label files
Copied 200/400 label files
Copied 250/400 label files
Copied 300/400 label files
Copied 350/400 label files
Copied 400/400 label files
Data copying completed!


In [8]:
from pathlib import Path

ROOT_PATH = Path("/content/local_data")

# List all files and folders in the directory
for item in ROOT_PATH.iterdir():
    print(item)

/content/local_data/train
/content/local_data/lists
/content/local_data/label
/content/local_data/processed
/content/local_data/train_processed_224
/content/local_data/val_processed_224


In [ ]:
# Run preprocessing and let it skip the corrupted files
!cd /content/drive/MyDrive/TransUNet-UNet3D-main

!python preprocess_imagecas.py \
    --image-dir /content/local_data/train \
    --label-dir /content/local_data/label \
    --out-train /content/local_data/train_processed_224 \
    --out-test /content/local_data/val_processed_224 \
    --lists-dir /content/local_data/lists \
    --split 0.2 \
    --seed 42

In [9]:
# Move the corrupted file out of the way
!mv /content/local_data/train/3.img.nii.gz /content/local_data/train/3.img.nii.gz.corrupted
!mv /content/local_data/label/3.label.nii.gz /content/local_data/label/3.label.nii.gz.corrupted

# Then rerun preprocessing
!python preprocess_imagecas.py \
    --image-dir /content/local_data/train \
    --label-dir /content/local_data/label \
    --out-train /content/local_data/train_processed_224 \
    --out-test /content/local_data/val_processed_224 \
    --lists-dir /content/local_data/lists \
    --split 0.2 \
    --seed 42

📦 399 volumes → train 320, val 79
✅ Done – slices in /content/local_data/train_processed_224 and /content/local_data/val_processed_224


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Run preprocessing using local data paths
os.chdir('/content/drive/MyDrive/TransUNet-UNet3D-main')

# Create output directories in local storage first
os.makedirs('/content/local_data/train_processed_224', exist_ok=True)
os.makedirs('/content/local_data/val_processed_224', exist_ok=True)
os.makedirs('/content/local_data/lists', exist_ok=True)

# Run preprocessing with local paths
!python preprocess_imagecas.py \
    --image-dir /content/local_data/train \
    --label-dir /content/local_data/label \
    --out-train /content/local_data/train_processed_224 \
    --out-test /content/local_data/val_processed_224 \
    --lists-dir /content/local_data/lists \
    --split 0.2 \
    --seed 42

In [11]:
# Check if data preprocessing was successful
train_dir = '/content/local_data/train_processed_224'
val_dir = '/content/local_data/val_processed_224'
lists_dir = '/content/local_data/lists'

print(f"Training slices: {len(os.listdir(train_dir))}")
print(f"Validation slices: {len(os.listdir(val_dir))}")
print(f"List files: {os.listdir(lists_dir)}")

# Check a sample file
import numpy as np
sample_files = [f for f in os.listdir(train_dir) if f.endswith('.npz')]
if sample_files:
    sample_data = np.load(f'{train_dir}/{sample_files[0]}')
    print(f"Sample data keys: {list(sample_data.keys())}")
    print(f"Image shape: {sample_data['image'].shape}")
    print(f"Label shape: {sample_data['label'].shape}")

Training slices: 85188
Validation slices: 20165
List files: ['train.txt', 'all.lst', 'test_vol.txt']
Sample data keys: ['image', 'label']
Image shape: (224, 224)
Label shape: (224, 224)


In [ ]:
!python test_imagecas_with_postprocessing.py random

In [ ]:
!python run_postprocessing_analysis.py --volume-filter "354.img.nii"